# 1. RAG

In [81]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
# !wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

!head documents.json

[
  {
    "course": "data-engineering-zoomcamp",
    "documents": [
      {
        "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
        "section": "General course-related questions",
        "question": "Course - When will the course start?"
      },
      {


In [90]:
import minsearch
import json
from pprint import pprint

In [8]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [82]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [91]:
pprint(documents[0])

{'course': 'data-engineering-zoomcamp',
 'question': 'Course - When will the course start?',
 'section': 'General course-related questions',
 'text': 'The purpose of this document is to capture frequently asked '
         'technical questions\n'
         'The exact day and hour of the course will be 15th Jan 2024 at 17h00. '
         "The course will start with the first  “Office Hours'' live.1\n"
         'Subscribe to course public Google Calendar (it works from Desktop '
         'only).\n'
         'Register before the course starts using this link.\n'
         'Join the course Telegram channel with announcements.\n'
         "Don’t forget to register in DataTalks.Club's Slack and join the "
         'channel.'}


In [84]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)
index.fit(documents)

In [87]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query, 
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost, 
        num_results=10
    )
    
    return results

In [97]:
q = 'the course has already started, can I still enroll?'
search_results = search(query=q)
pprint(search_results[:2])

[{'course': 'data-engineering-zoomcamp',
  'question': 'Course - Can I still join the course after the start date?',
  'section': 'General course-related questions',
  'text': "Yes, even if you don't register, you're still eligible to submit "
          'the homeworks.\n'
          'Be aware, however, that there will be deadlines for turning in the '
          "final projects. So don't leave everything for the last minute."},
 {'course': 'data-engineering-zoomcamp',
  'question': 'Course - Can I follow the course after it finishes?',
  'section': 'General course-related questions',
  'text': 'Yes, we will keep all the materials after the course finishes, so '
          'you can follow the course at your own pace after it finishes.\n'
          'You can also continue looking at the homeworks and continue '
          'preparing for the next cohort. I guess you can also start working '
          'on your final capstone project.'}]


# 2. LLM Prompt

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

import google.generativeai as genai

api_token = os.getenv("API_KEY")

genai.configure(api_key=api_token)

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
q = 'the course has already started, can I still enroll?'

In [4]:
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content(q)
print(response.text)

It depends on the course and the institution offering it.  Some courses allow late enrollment, while others do not.  To know for sure, you should:

* **Check the course syllabus or website:**  Look for information about late enrollment deadlines or contact information for the instructor or course administrator.
* **Contact the instructor or course administrator directly:** This is the best way to find out if late enrollment is possible and what the process is.

Don't assume you can't enroll just because the course has started.  It's always worth checking.



# 3. RAG and LLM

In [142]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""
    for result in search_results:
        context += f"section: {result["section"]} \nquestion: {result["question"]} \nanswer: {result["text"]}\n\n"

    full_prompt = prompt_template.format(question=query, context=context)

    return full_prompt

In [155]:
def llm(prompt):
    # Create the model
    generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
    }

    model = genai.GenerativeModel(model_name="gemini-2.0-flash-exp",
                                  #generation_config=generation_config
                                  )
    response = model.generate_content(prompt)
    return response.text

In [156]:
def rag(query):
    search_results = search(query=query)
    full_prompt = build_prompt(query=query, search_results=search_results)
    answer = llm(prompt=full_prompt)
    return answer

In [157]:
print(rag('The course already started, can I still join?'))

Yes, you can still submit homework even if you don't register for the course. Keep in mind there are deadlines for the final projects.

